In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


For the given sentence, "My husband, daughter & I got there at 2:30, and the food is amazing!!!    (5 star)," the resulting list of words after preprocessing should be: ['husband', 'daughter', 'got', '2:30', '&', 'food', 'amazing', '5', 'star'].

Please note that the colon in "2:30" is not removed during this process.

In [ ]:
import argparse
import json
import re
import gc
import heapq

# A. The number of distinct users who have written the reviews
# B. The number of reviews in a given month, m
# C. Top n users who have the largest number of reviews and its count
# D. The total word count for each year
# E. The average length of reviews in terms of word count
# F. Top i frequent words in the review text




def main(args):
    review_list = []
    review_user_id = []
    review_user_dic = {}
    dstinct_user = 0
    top_reviewer = []
    date_count = 0
    year_review = {}
    yearly_word_count=[]
    stop_word_list = []
    review_count = 0
    review_len_count = 0
    avg_review_len= 0
    review_word_dic = {}
    frequent_word = []
    frequent_words_only = []

    with open(args.stopwords, 'r', encoding='utf-8', errors='replace') as stop_word:
        for i in stop_word:
            word=i.strip().lower()
            stop_word_list.append(word)

    # print("I worked on stopwords")


    stop_word_list=stop_word_list+["(", "[", ",", ".", "!", "?", ":", ";", "]",")"]


    with open(args.input_file, 'r', encoding='utf-8', errors='replace') as review:
        for i in review:
            row = json.loads(i.strip())
            review_list.append([row["text"], row["date"]])
            review_user_id.append(row["user_id"])
            # B
            date = row["date"].split(" ")[0].split("-")[1]
            if date == str(args.m).zfill(2):
                date_count += 1

    # print("I crossed B")

    # A
    dstinct_user = len(set(review_user_id))

    # print("I crossed A")

    # C
    for user_id in review_user_id:
        if user_id in review_user_dic:
            review_user_dic[user_id] += 1
        else:
            review_user_dic[user_id] = 1

    del review_user_id
    gc.collect()

    # print("I am in C")

    top_reviewer = heapq.nlargest(args.n, review_user_dic.items(), key=lambda item: item[1])

    del review_user_dic
    gc.collect()

    # print("I crossed C")

    # D & E
    for dated_review in review_list:
        year = dated_review[1].split(" ")[0].split("-")[0]
        filtered_words = []
        for word in dated_review[0].split():
            word = re.sub(r'(?<!\d):(?!\d)|[^a-zA-Z0-9:&]', '', word).strip().lower()
            if word and word not in stop_word_list:
                filtered_words.append(word)

        print(filtered_words)
        filtered_review_len = len(filtered_words)
        review_count +=1
        review_len_count += filtered_review_len

        # D:
        if year in year_review:
            year_review[year] += filtered_review_len
        else:
            year_review[year] = filtered_review_len

        # F
        for word in filtered_words:
            if word in review_word_dic:
                review_word_dic[word] += 1
            else:
                review_word_dic[word] = 1


    # print("I am in D&E&F")

    # D continuing
    yearly_word_count = sorted(year_review.items(), key=lambda item: (-item[1], item[0]))

    del year_review
    gc.collect()

    # print("I crossed D")

    # E continuing
    avg_review_len= review_len_count/review_count

    del review_list
    gc.collect()

    # print("I crossed E")

    # F continuing
    frequent_word = heapq.nlargest(args.i, review_word_dic.items(), key=lambda x: x[1])

    for item in frequent_word:
        frequent_words_only.append(item[0])

    del review_word_dic
    gc.collect()

    # print("I crossed F")

    data ={"A": dstinct_user, "B": date_count, "C": top_reviewer, "D": yearly_word_count, "E": avg_review_len, "F": frequent_words_only}

    json_data = json.dumps(data)
    print(json_data)

    with open(args.output_file, 'w') as json_data:
        json.dump(data, json_data)




if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='HW1T1')
    parser.add_argument('--input_file', type=str, default='./review_test1.json', help='input file')
    parser.add_argument('--output_file', type=str, default='./hw1t1.json', help='output file')
    parser.add_argument('--stopwords', type=str, default='./data/stopwords', help='stopword file')
    parser.add_argument('--m', type=int, default=12, help='month')
    parser.add_argument('--n', type=int, default=1, help='top n users')
    parser.add_argument('--i', type=int, default=1, help='top i frequent words')

    args = parser.parse_args()
    main(args)

In [4]:
import argparse
import json
import re
import gc
import heapq
from tqdm.notebook import tqdm

In [5]:
review_list = []
review_user_id = []
review_user_dic = {}
dstinct_user = 0
top_reviewer = []
date_count = 0
year_review = {}
yearly_word_count=[]
stop_word_list = []
review_count = 0
review_len_count = 0
avg_review_len= 0
review_word_dic = {}
frequent_word = []
frequent_words_only = []

In [ ]:

with open("/content/drive/MyDrive/00000/stopwords", 'r', encoding='utf-8', errors='replace') as stop_word:
    for i in stop_word:
        word=i.strip().lower()
        stop_word_list.append(word)


stop_word_list=stop_word_list+["(", "[", ",", ".", "!", "?", ":", ";", "]",")"]

In [ ]:
"""main = "My husband, daughter & I got there at 2:30, and the food is amazing!!! (5 star),".split(" ")
d= []


for i  in main:
  if i.lower() not in stop_word_list:
    d.append(''.join(char for char in i if char not in '''!()-[]{};'"\,<>./?*_~'''))


d"""

'main = "My husband, daughter & I got there at 2:30, and the food is amazing!!! (5 star),".split(" ")\nd= []\n\n\nfor i  in main:\n  if i.lower() not in stop_word_list:\n    d.append(\'\'.join(char for char in i if char not in \'\'\'!()-[]{};\'"\\,<>./?*_~\'\'\'))\n\n\nd'

In [ ]:
['husband', 'daughter', 'got', '2:30', '&', 'food', 'amazing', '5', 'star']

['husband', 'daughter', 'got', '2:30', '&', 'food', 'amazing', '5', 'star']

In [ ]:
with open("/content/drive/MyDrive/00000/review.json", 'r', encoding='utf-8', errors='replace') as review:
    for i in review:
        row = json.loads(i.strip())
        review_list.append([row["text"], row["date"]])
        review_user_id.append(row["user_id"])
        # B
        date = row["date"].split(" ")[0].split("-")[1]
        #if date == str(args.m).zfill(2):
        if date == str(12).zfill(2):
            date_count += 1

In [6]:
from tqdm.notebook import tqdm

all_rows = []

with open("/content/drive/MyDrive/00000/review.json", 'r', encoding='utf-8', errors='replace') as review:
    for i in tqdm(review):
        row = json.loads(i.strip())
        all_rows.append(row)


0it [00:00, ?it/s]

In [13]:
non_dict = []
dic = []
for i in tqdm(all_rows):
  if str(type(i)) != "<class 'dict'>":
    non_dict.append(i)
    break
  else:
    dic.append(i)

  0%|          | 0/1151625 [00:00<?, ?it/s]

In [14]:
len(non_dict)

0

In [15]:
len(dic)

1151625

In [ ]:
# print("I crossed B")

# A
dstinct_user = len(set(review_user_id))

# print("I crossed A")

# C
for user_id in tqdm(review_user_id):
    if user_id in review_user_dic:
        review_user_dic[user_id] += 1
    else:
        review_user_dic[user_id] = 1

del review_user_id
gc.collect()

# print("I am in C")

top_reviewer = heapq.nlargest(1, review_user_dic.items(), key=lambda item: item[1])

del review_user_dic
gc.collect()

# print("I crossed C")




  0%|          | 0/1151625 [00:00<?, ?it/s]

0

In [ ]:


all_word = []

# D & E
for dated_review in tqdm(review_list):
    year = dated_review[1].split(" ")[0].split("-")[0]
    filtered_words = []
    for word in dated_review[0].split():
        word = re.sub(r'(?<!\d):(?!\d)|[^a-zA-Z0-9:&]', '', word).strip().lower()
        if word and word not in stop_word_list:
            filtered_words.append(word)
            all_word.append(word)

    #print(filtered_words)
    filtered_review_len = len(filtered_words)
    review_count +=1
    review_len_count += filtered_review_len

    # D:
    if year in year_review:
        year_review[year] += filtered_review_len
    else:
        year_review[year] = filtered_review_len

    # F
    for word in filtered_words:
        if word in review_word_dic:
            review_word_dic[word] += 1
        else:
            review_word_dic[word] = 1


print("I am in D&E&F")

# D continuing
yearly_word_count = sorted(year_review.items(), key=lambda item: (-item[1], item[0]))

del year_review
gc.collect()

print("I crossed D")

# E continuing
avg_review_len= review_len_count/review_count

del review_list
gc.collect()

print("I crossed E")


  0%|          | 0/1151625 [00:00<?, ?it/s]

I am in D&E&F
I crossed D
I crossed E


In [ ]:
filtered_words

['soon',
 'got',
 'got',
 'amazing',
 'service',
 'staff',
 'super',
 'nice',
 'everyone',
 'looked',
 'like',
 'happy',
 'unusual',
 'see',
 'restaurants',
 'husband',
 'ordered',
 'brisket',
 'plate',
 'option',
 'moist',
 'dry',
 'chose',
 'moist',
 'plate',
 'automatically',
 'comes',
 'cornbread',
 'isnt',
 'best',
 'thing',
 'ive',
 'ever',
 'cornbread',
 'different',
 'actually',
 'pieces',
 'corn',
 'inside',
 'dry',
 'definitely',
 'taste',
 'smoked',
 'flavor',
 'brisket',
 'bit',
 'bland',
 'add',
 'whatever',
 'bbq',
 'sauce',
 'youd',
 'please',
 'loved',
 'kc',
 'sauce',
 'nice',
 'spicy',
 'kick',
 'didnt',
 'care',
 'much',
 '5',
 'sauces',
 'preference',
 'honestly',
 'dont',
 'think',
 'pricing',
 'bad',
 'compared',
 'would',
 'usually',
 'spend',
 'bbq',
 'dinner',
 'south',
 'definitely',
 'try',
 'place',
 'see',
 'consistent',
 'food',
 'since',
 'opened',
 'recently']

In [ ]:
# F continuing
frequent_word = heapq.nlargest(args.i, review_word_dic.items(), key=lambda x: x[1])

for item in frequent_word:
    frequent_words_only.append(item[0])

del review_word_dic
gc.collect()

# print("I crossed F")

data ={"A": dstinct_user, "B": date_count, "C": top_reviewer, "D": yearly_word_count, "E": avg_review_len, "F": frequent_words_only}

json_data = json.dumps(data)
print(json_data)

with open(args.output_file, 'w') as json_data:
    json.dump(data, json_data)


